In [1]:
import json
import re

In [2]:
# Load few-shot data
file_location = "/Users/thyag/Desktop/codes/chain-of-thought/dataset/arithmetic reasoning/gsm8k/base/gsm8k-base-gemini-2-0-flash.json"
with open(file_location, "r") as f:
    few_shot_data = json.load(f)

In [26]:
from openai import OpenAI
import json
import re
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def normalize_numeric(value_str):
    try:
        return str(float(value_str))
    except ValueError:
        return value_str

def openai_extract_final_answer(text):
    """
    Use the OpenAI API to extract the main answer from the given text.
    """
    # Note: The 'responses.create' endpoint is hypothetical.
    # The standard is 'chat.completions.create'.
    # This function is kept as-is from your example.
    # If using a standard model like gpt-4o-mini, the call would look like this:
    #
    # response = client.chat.completions.create(
    #      model="gpt-4o-mini",
    #      messages=[
    #          {"role": "system", "content": "Extract the final answer from the text. Only provide the final answer and nothing else. Do not include any reasoning or explanation."},
    #          {"role": "user", "content": text}
    #      ]
    # )
    # return response.choices[0].message.content
    
    # Using the hypothetical endpoint from your code
    response = client.responses.create(
         model="gpt-4o-mini-2024-08-06",
         instructions="Extract the final answer from the text. Only provide the final answer and nothing else. Do not include any reasoning or explanation.",
         input=text,
    )
    return response.output_text

def extract_final_answer(text):
    """
    Extract the final answer from text in various formats.
    Works with numerical answers, multiple choice, true/false, and text answers.
    For ambiguous cases, including empty input, falls back to using the OpenAI API.
    """
    if text and text.strip():
        cleaned_text = text.strip()
        cleaned_text = re.sub(r'[\*\_`]', '', cleaned_text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

        # Numeric patterns: ensure we only match when context indicates explicit "answer"
        patterns = [
            r'answer\s*(?:is|[:=])\s*\$?\\?boxed\{?\$?([\d\.]+)\$?\}?',
            r'answer\s*(?:is|[:=])\s*([\d\.]+)',
            r'final\s*(?:answer|result)\s*(?:[:=])\s*([\d\.]+)',
            r'result\s*(?:is)?\s*([\d\.]+)',
            r'equals\s+([\d\.]+)',
            r'we get\s+([\d\.]+)',
            r'thus[,:\s]+([\d\.]+)',
            r'hence[,:\s]+([\d\.]+)',
            r'([\d\.]+)\s+is\s+the\s+final\s+answer',
        ]

        for pattern in patterns:
            match = re.search(pattern, cleaned_text, re.IGNORECASE)
            if match:
                return normalize_numeric(match.group(1))

        mc_match = re.search(r'answer\s*(?:is)?\s*([A-D])[\.:\)]', cleaned_text, re.IGNORECASE)
        if mc_match:
            return mc_match.group(1).upper()

        yn_match = re.search(r'answer\s*(?:is)?\s*(yes|no|true|false)', cleaned_text, re.IGNORECASE)
        if yn_match:
            return yn_match.group(1).lower()

        text_match = re.search(r'answer\s*(?:is|[:=])\s*"([^"]+)"', cleaned_text, re.IGNORECASE)
        if text_match:
            return text_match.group(1).strip()

        lines = [line.strip() for line in text.split('\n') if line.strip()]
        if lines:
            last_line = lines[-1]
            for prefix in ['Answer:', 'Therefore,', 'Thus,']:
                if last_line.startswith(prefix):
                    answer_part = last_line[len(prefix):].strip()
                    num_match = re.search(r'([\d\.]+)', answer_part)
                    if num_match:
                        return normalize_numeric(num_match.group(1))
                    return answer_part
            match = re.search(r'([\d\.]+)\s*$', last_line)
            if match:
                return normalize_numeric(match.group(1))

    try:
        input_for_api = text or "" or "."
        api_result = openai_extract_final_answer(input_for_api)
        return api_result
    except Exception:
        return "API_ERROR"

# Run the fixed function on the example data
for i, example in enumerate(few_shot_data):
    pred = extract_final_answer(example["answer_text"])
    print(f"Prediction: {pred}\n")

Prediction: 72.0

Prediction: 10.0

Prediction: .

Prediction: .

Prediction: .

Prediction: .

Prediction: .

Prediction: 16.0

Prediction: .

Prediction: .

Prediction: .

Prediction: .

Prediction: .

Prediction: .

Prediction: 5.0

Prediction: .

Prediction: .

Prediction: 43.0

Prediction: 16.0

Prediction: .



In [24]:
for i, example in enumerate(few_shot_data):
    pred = extract_final_answer(example["answer_text"])
    print(f"Example {i+1}: {pred}")

Example 1: 72
Example 2: 10
Example 3: 
Example 4: 
Example 5: 
Example 6: 
Example 7: 
Example 8: 16
Example 9: 
Example 10: 
Example 11: 
Example 12: 
Example 13: 
Example 14: 
Example 15: 5
Example 16: 
Example 17: 
Example 18: 43
Example 19: 16
Example 20: 


In [ ]:
for i, example in enumerate(few_shot_data):
    pred = extract_final_answer(example["answer_text"])
    print(f"Example {i+1}: {pred}")

Example 1: 72
Example 2: 10
Example 3: 
Example 4: 
Example 5: 
Example 6: 
Example 7: 
Example 8: 16
Example 9: 
Example 10: 
Example 11: 
Example 12: 
Example 13: 
Example 14: 
Example 15: 5
Example 16: 
Example 17: 
Example 18: 43
Example 19: 16
Example 20: 


In [5]:
for i, example in enumerate(few_shot_data):
    pred = extract_final_answer(example["original_answer"])
    print(f"Example {i+1}: {pred}")

Example 1: 72
Example 2: 10
Example 3: 5
Example 4: 42
Example 5: 624
Example 6: 35
Example 7: 48
Example 8: 16
Example 9: 41
Example 10: 990
Example 11: 121
Example 12: 5
Example 13: 85
Example 14: 35
Example 15: 5
Example 16: 448000
Example 17: 800
Example 18: 43
Example 19: 16
Example 20: 16
